In [ ]:
import pandas as pd
import os
import numpy as np
import csv

In [ ]:
path =  r"C:\Users\mme\Desktop\BTech FYP _ Tamaghna\Data\First Data set\HR.xlsx"  #D:\Python Codes\Recrystallization FYP\HR.xlsx
#print(path)
df = pd.read_excel(path)
# print(df)
df = df.to_numpy()

stepsize = df[1,3] - df[0,3]

df[:,3] = (df[:, 3] / (stepsize)).astype(int)
df[:,4] = (df[:, 4] / (stepsize)).astype(int)

Converting the numpy array 

In [ ]:
theta_m = 15 # 15 degree is the critical value for misorientation

r,c = int(np.max(df[:,3])),int(np.max(df[:,4]))
# print(r,c)
s = np.zeros((r+1,c+1,3))
for i in df:
    # print(i)
    s[int(i[3])][int(i[4])] = i[0:3]

Defining the function G

In [ ]:
def g(phi1,phi,phi2):
    g_one = np.array([ [np.cos(phi1), np.sin(phi1), 0] ,[-np.sin(phi1), np.cos(phi1), 0] ,[0,0,1]])
    g_two = np.array([ [np.cos(phi2), np.sin(phi2), 0] ,[-np.sin(phi2), np.cos(phi2), 0] ,[0,0,1]])
    g = np.array([ [1,0,0], [0, np.cos(phi), np.sin(phi)] ,[0, -np.sin(phi), np.cos(phi)]])

    b = np.matmul(g,g_one)
    c = np.matmul(g_two,b)

    return c
print(g(s[1,1,0],s[1,1,1],s[1,1,2]))

In [ ]:
G = np.zeros((r + 1, c + 1, 2, 3, 3))

for x in range(0,r+1):
    for y in range(0,c+1):
        G[x,y]= [g(s[x,y,0] , s[x,y,1] , s[x,y,0]), np.linalg.inv(g(s[x,y,0] , s[x,y,1] , s[x,y,0]))]

# print(g(s[1,1,0],s[1,1,1],s[1,1,2]))


In [ ]:
print(G)

In [ ]:
def theta(del_g): 
    symmetry_matrix = np.array([
    [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    [[0, 0, 1], [1, 0, 0], [0, 1, 0]],
    [[0, 1, 0], [0, 0, 1], [1, 0, 0]],
    [[0, -1, 0], [0, 0, 1], [-1, 0, 0]],
    [[0, -1, 0], [0, 0, -1], [1, 0, 0]],
    [[0, 1, 0], [0, 0, -1], [-1, 0, 0]],
    [[0, 0, -1], [1, 0, 0], [0, -1, 0]],
    [[0, 0, -1], [-1, 0, 0], [0, 1, 0]],
    [[0, 0, 1], [-1, 0, 0], [0, -1, 0]],
    [[-1, 0, 0], [0, 1, 0], [0, 0, -1]],
    [[-1, 0, 0], [0, -1, 0], [0, 0, 1]],
    [[1, 0, 0], [0, -1, 0], [0, 0, -1]],
    [[0, 0, -1], [0, -1, 0], [-1, 0, 0]],
    [[0, 0, 1], [0, -1, 0], [1, 0, 0]],
    [[0, 0, 1], [0, 1, 0], [-1, 0, 0]],
    [[0, 0, -1], [0, 1, 0], [1, 0, 0]],
    [[-1, 0, 0], [0, 0, -1], [0, -1, 0]],
    [[1, 0, 0], [0, 0, -1], [0, 1, 0]],
    [[1, 0, 0], [0, 0, 1], [0, -1, 0]],
    [[-1, 0, 0], [0, 0, 1], [0, 1, 0]],
    [[0, -1, 0], [-1, 0, 0], [0, 0, -1]],
    [[0, 1, 0], [-1, 0, 0], [0, 0, 1]],
    [[0, 1, 0], [1, 0, 0], [0, 0, -1]],
    [[0, -1, 0], [1, 0, 0], [0, 0, 1]]])
    
    min_value = np.inf

    for i in symmetry_matrix:
        n_val = (np.arccos((np.trace(np.matmul( i, del_g) ) - 1) / 2))
        print(n_val)
        if  n_val < min_value:
            min_value = n_val
     
    return min_value

In [ ]:
theta(np.matmul(G[1,1,0],G[1,2,1]))

In [ ]:
def stored_energy(theta):
    #Read schokely equation
    s = np.degrees(theta)
    
    if s < 0.1:
        return 0

    if s > theta_m:
        return 10/2
    else:
        return (10*(s/theta_m)*(1-np.log(s/theta_m)))/2
    

In [ ]:
#Basic Output of inner cells

stored_energy_values = np.zeros((r + 1, c + 1, 1))

for x in range(1,r):
    for y in range(1,c,1):
        for i in range(-1,2,1):
            for j in range(-1,2,1):
                stored_energy_values[x,y,0] = stored_energy_values[x,y,0] + stored_energy(theta(np.matmul(G[x,y,0],G[x+i,y+j,1])))
                
        stored_energy_values[x,y,0] = (stored_energy_values[x,y,0] - stored_energy(theta(np.matmul(G[x,y,0],G[x,y,1]))))/8        
        

In [ ]:
f = open("energy_values1.txt", "w" )
f.write("X \t Y \t E \n")
for x in range(len(stored_energy_values)):
    for y in range(len(stored_energy_values[x])):
        f.write("%s \t %s \t %s \n"%(x,y,stored_energy_values[x,y,0]))

In [ ]:
with open('students.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["X", "Y", "E"])
    for x in range(len(stored_energy_values)):
        for y in range(len(stored_energy_values[x])):
            writer.writerow([x,y,stored_energy_values[x,y,0]])

